In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

## Implement ML functions

In [3]:
def compute_cost(y, tx, w):
    return compute_cost_MSE(y,tx,w)
    
def compute_cost_MSE(y,tx,w):
    e=y-(tx @ w)
    return (1/(2*y.shape[0]))*(e.T @ e)
def compute_cost_MAE(y,tx,w):
    e=y-(tx @ w)
    return (1/y.shape[0])*np.absolute(e).sum()

In [4]:
def compute_gradient_MSE(y,tx,w):
    """Compute the gradient."""
    e=y-(tx @ w)
    return -1/y.shape[0]*(tx.T @ e)

In [5]:
def general_gradient_descent(y, tx, initial_w, max_iters, gamma, grad_function, cost_function):
    """Gradient descent algorithm who work with arbitrary gradient and cost function
    grad and cost function should take y,tw and w as parameter and return resêctivly the gradient vector and the scalar error"""
    # Define parameters to store w and loss
    ws = [initial_w.ravel()]
    losses = []
    w = initial_w.ravel()
    for n_iter in range(max_iters):
        #compute gradient and loss
        gradient=grad_function(y,tx,w)
        loss=cost_function(y,tx,w)
        if n_iter % 1 == 0:
            print("Current iteration={i}, the loss={l}".format(i=n_iter, l=loss))
        #update w by gradient
        w=w-gamma*gradient
        
        # store w and loss
        ws.append(np.copy(w))
        losses.append(loss)
        #print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
        #     bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses[-1], ws[-1]

In [6]:
def general_stochastic_gradient_descent(y, tx, initial_w, batch_size, max_iters, gamma, stock_grad_function, cost_function):
    """Gradient descent algorithm who work with arbitrary gradient and cost function
    grad and cost function should take y,tw and w as parameter and return respectivly the gradient vector and the scalar error"""
    
    # implement stochastic gradient descent.
    ws = [initial_w.ravel()]
    losses = []
    w = initial_w.ravel()
    
    minibatchs = batch_iter(y, tx, batch_size, num_batches=math.floor(y.shape[0]/batch_size))
    for n_iter in range(0,max_epochs):
        
        # compute gradient and loss
        minibatch=minibatchs.__next__()
        gradient=stock_grad_function(minibatch[0],minibatch[1],w)
        loss=cost_function(y,tx,w)
        
        # update w by gradient
        w=w-gamma*gradient
        # store w and loss
        ws.append(np.copy(w))
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses[-1], ws[-1]

In [7]:
def least_squares_GD(y, tx, gamma, max_iters):
    return general_gradient_descent(y,tx,np.zeros((tx.shape[1], 1)).ravel(),max_iters,gamma,compute_gradient_MSE,compute_cost_MSE)

#general_gradient_descent(y,tX,np.zeros((tX.shape[1], 1)),20,0.000003,compute_gradient_MSE,compute_cost_MSE)


In [8]:
def least_squares_SGD(y, tx, gamma, max_iters):
    batch_size = y.shape[0]//2
    return general_stochastic_gradient_descent(y,tx,np.zeros((tx.shape[1], 1)),batch_size,max_iters,gamma,compute_gradient_MSE,compute_cost_MSE)

In [9]:
def least_squares(y, tx):
    w = np.linalg.solve(tx.T.dot(tx),tx.T.dot(y)) #return best weight
    return compute_cost_MSE(y, tx, w), w

In [10]:
def ridge_regression(y, tx, lamb):
    """implement ridge regression."""
    
    lambp = lamb*(2*tx.shape[0])
    #return np.linalg.inv(tx.T.dot(tx)+lambp*np.eye(tx.shape[1])).dot(tx.T).dot(y)
    w =  np.linalg.solve(tx.T.dot(tx)+lambp*np.eye(tx.shape[1]),tx.T.dot(y))
    return compute_cost_MSE(y, tx, w), w

### Logistic

In [11]:
def sigmoid(t):
    """apply sigmoid function on t."""
    z = np.exp(t)
    return z/(1+z)

def compute_loss_logistic(y, tx, w, lambda_ = 0):
    """compute the cost by negative log likelihood."""
    clip = np.clip(tX @ w, -700, 700)
    if lambda_ == 0:
        return np.sum(np.log(1+np.exp(clip))-y*(tx @ w))
    else:
        return np.sum(np.log(1+np.exp(clip))-y*(tx @ w)) + lambda_*np.sum(w*w) #or + lambda_*w.T*w
    #return -np.sum(np.log(1+np.exp(tx @ w))-y*(tx @ w))
    
def compute_gradient_sigmoid(y, tx, w, lambda_ = 0):
    """compute the gradient of loss."""
    clip = np.clip(tX @ w, -700, 700)
    if lambda_ == 0:
        return tx.T.dot(sigmoid(clip)-y)
    else:
        return tx.T.dot(sigmoid(clip)-y) + lambda_*2*w

In [12]:
def logistic_regression(y, tx, gamma ,max_iters):
    print("fuck")
    ok = general_gradient_descent(y,tx,np.zeros((tx.shape[1], 1)),max_iters,gamma,compute_gradient_sigmoid,compute_loss_logistic)
    return ok

In [13]:
def reg_logistic_regression(y, tx, lambda_ , gamma, max_iters):
    gradf = partial(compute_gradient_sigmoid,lambda_ = lambda_)
    costf = partial(compute_loss_logistic, lambda_ = lambda_)
    return general_gradient_descent(y,tx,np.zeros((tx.shape[1], 1)),max_iters,gamma,gradf,costf)

## Prediction Formula

In [21]:
w=np.zeros((tX.shape[1], 1))#.ravel()
#ok = general_gradient_descent(y,tX,w,50,0.00000000003,compute_gradient_sigmoid,compute_loss_logistic)
ok2 = logistic_regression(y,tX,0.00000000003,20)

fuck
Current iteration=0, the loss=173286.79513998624
Current iteration=1, the loss=-2725323.636153825
Current iteration=2, the loss=-3971997.8492310946
Current iteration=3, the loss=-5216589.725502855
Current iteration=4, the loss=-6459791.38545703
Current iteration=5, the loss=-7702040.284123596
Current iteration=6, the loss=-8943607.951261211
Current iteration=7, the loss=-10184668.000161488
Current iteration=8, the loss=-11425336.560482027
Current iteration=9, the loss=-12665694.868688703
Current iteration=10, the loss=-13905802.001533179
Current iteration=11, the loss=-15145702.310699685
Current iteration=12, the loss=-16385429.957667599
Current iteration=13, the loss=-17625011.804495726
Current iteration=14, the loss=-18864469.33292817
Current iteration=15, the loss=-20103819.964558963
Current iteration=16, the loss=-21343077.99692411
Current iteration=17, the loss=-22582255.284399487
Current iteration=18, the loss=-23821361.744191263
Current iteration=19, the loss=-25060405.7392

In [ ]:
w=np.zeros((tX.shape[1], 1)).ravel()
y2 = y.copy()
y2[np.where(y==-1)] = 0
test = np.clip(tX @ w, -700, 700)
zbub = compute_loss_logistic(y2,tX,w,0.03)
zbub2 = compute_gradient_sigmoid(y2,tX,w,0.03)
tX2 = tX.copy()
tX2[np.where(tX2==-999)] = 0
e=y-(tX @ w)
print(np.log(1+np.exp(test))-y*(tX @ w))
#ok = general_gradient_descent(y2,tX2,w,50,0.00000000003,compute_gradient_sigmoid,compute_loss_logistic)
ok = least_squares_GD(y2,tX2,0.00000000003,20)
#ok = general_gradient_descent(y2,tX,w,20,0.0000003,compute_gradient_MSE,compute_cost_MSE)
#ok = general_gradient_descent(y2,tX,w,10,-0.003,compute_gradient_MSE,compute_cost_MSE)
#print((tX @ w).shape,zbub,zbub2,ok)
#print(txs.shape,y.shape[1])
#compute_loss_logistic(y,test,w)

#a = compute_loss_logistic(y,tX,w)
#print(txs,sigmoid(-700))

[ 0.69314718  0.69314718  0.69314718 ...,  0.69314718  0.69314718
  0.69314718]


In [32]:
# _ ,weights = least_squares(y,tX)
losss,weights = least_squares(y,tX)

In [33]:
print(losss,weights)

0.339686809477 [  8.03494352e-05  -7.20202266e-03  -6.05417274e-03  -5.47559078e-04
  -1.93874687e-02   4.73451613e-04  -2.60379058e-02   3.25106299e-01
  -3.80780000e-05  -2.72788699e+00  -2.21220141e-01   9.50794097e-02
   6.40351609e-02   2.73614667e+00  -3.31801090e-04  -9.54325141e-04
   2.74090341e+00  -5.34165279e-04   9.73498912e-04   3.69225050e-03
   3.54487164e-04  -5.43344618e-04  -3.30448035e-01  -1.40800497e-03
   8.31432840e-04   1.02117276e-03  -1.68047418e-03  -5.83664793e-03
  -1.11088004e-02   2.72834692e+00]


## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '../test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '../submission.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [ ]:
def compute_gradient(y, tx, w):
    return compute_gradient_MSE(y,tx,w)

def compute_gradient_MSE(y,tx,w):
    """Compute the gradient."""
    e=y-(tx @ w)
    return -1/y.shape[0]*tx.T @ e

def gradient_descent(y, tx, initial_w, max_iters, gamma): 
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        #compute gradient and loss
        gradient=compute_gradient(y,tx,w)
        loss=compute_cost(y,tx,w)
        
        #update w by gradient
        w=w-gamma*gradient
        
        # store w and loss
        ws.append(np.copy(w))
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses[-1], ws

In [ ]:
def compute_stoch_gradient(y, tx, w):
    """Compute a stochastic gradient for batch data."""

    return compute_gradient(y,tx,w)

import math

def stochastic_gradient_descent(y, tx, initial_w, batch_size, max_epochs, gamma):
    """Stochastic gradient descent algorithm."""
    
    # implement stochastic gradient descent.
    ws = [initial_w]
    losses = []
    w = initial_w
    
    minibatchs = batch_iter(y, tx, batch_size, num_batches=math.floor(y.shape[0]/batch_size))
    for n_iter in range(0,max_epochs):
        
        # compute gradient and loss
        minibatch=minibatchs.__next__()
        gradient=compute_stoch_gradient(minibatch[0],minibatch[1],w)
        loss=compute_cost(y,tx,w)
        
        # update w by gradient
        w=w-gamma*gradient
        # store w and loss
        ws.append(np.copy(w))
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses, ws